### 海淀苏州街数据爬取

##### 0. 一些准备

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time
import re

In [14]:
def click_haidian_suzhouqiao(url):
    """点击海淀区和苏州桥片区"""
    try:
        # 等待页面加载并点击北京
        driver.get(url)
        
        # 点击海淀区
        haidian_link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "海淀")))
        haidian_link.click()
        print("已点击海淀区")
        
        # 点击苏州桥片区
        suzhouqiao_link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "苏州桥")))
        suzhouqiao_link.click()
        print("已点击苏州桥片区")
        
    except Exception as e:
        print(f"点击区域时出错: {e}")

def scrape_listings(element_list, npage_button, func, max_pages=20):
    """爬取房源列表"""
    all_listings = []
    
    try:
        for page in range(1, max_pages + 1):
            print(f"正在爬取第 {page} 页...")
            
            # 获取当前页所有房源元素
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, element_list)))
            listing_elements = driver.find_elements(By.CSS_SELECTOR, element_list)
            print(f"找到 {len(listing_elements)} 个房源")
            
            page_listings = 0
            for i, listing_element in enumerate(listing_elements):
                try:
                    listing_data = func(listing_element)
                    if listing_data:
                        all_listings.append(listing_data)
                        page_listings += 1
                    
                except Exception as e:
                    print(f"处理第 {page} 页第 {i+1} 条房源时出错: {e}")
            
            print(f"第 {page} 页收集完成，共 {page_listings} 条房源")
            
            # 尝试翻到下一页
            if page < max_pages:
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, npage_button)
                    driver.execute_script("arguments[0].click();", next_button)
                    print("成功点击下一页")
                    time.sleep(1) # 太快似乎会引起反爬
                    
                    # 等待新页面加载
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, element_list)))
                    
                except NoSuchElementException:
                    print("没有找到下一页，爬取结束")
                    break     
    except Exception as e:
        print(f"爬取过程中出错: {e}")
        import traceback
        traceback.print_exc()
    
    return all_listings                    

##### 1. 爬取房价数据，存储在'Housing_Price_Suzhouqiao.csv'

In [15]:
def collect_PRICE_listing_from_list(listing_element):
    """从列表页元素直接收集数据"""
    listing_data = {}
    
    try:
        # 标题
        try:
            title_element = listing_element.find_element(By.CSS_SELECTOR, 'h4 a span.tit_shop')
            listing_data['标题'] = title_element.text
        except NoSuchElementException:
            listing_data['标题'] = ''
        
        # 总价
        try:
            total_price_element = listing_element.find_element(By.CSS_SELECTOR, 'dd.price_right span.red b')
            listing_data['总价(万元)'] = float(total_price_element.text)
        except NoSuchElementException:
            listing_data['总价(万元)'] = None
        
        # 单价
        try:
            unit_price_element = listing_element.find_element(By.CSS_SELECTOR, 'dd.price_right span:not(.red)')
            unit_price_text = unit_price_element.text
            cleaned_unit_price_text = re.sub(r'[^\d.]', '', unit_price_text)
            listing_data['单价(元/平方米)'] = float(cleaned_unit_price_text)
        except NoSuchElementException:
            listing_data['单价(元/平方米)'] = None
        
        # 户型、面积、楼层、朝向、建成年份
        try:
            tel_shop_element = listing_element.find_element(By.CSS_SELECTOR, 'p.tel_shop')
            tel_shop_text = tel_shop_element.text
            parts = [part.strip() for part in tel_shop_text.split('|')]
            
            # 户型
            if len(parts) > 0:
                listing_data['户型'] = parts[0]
            
            # 面积
            if len(parts) > 1:
                area_text = re.sub(r'[^\d.]', '', parts[1])
                listing_data['面积'] = float(area_text)
            
            # 楼层
            if len(parts) > 2:
                listing_data['楼层'] = parts[2]
            
            # 朝向
            if len(parts) > 3:
                listing_data['朝向'] = parts[3]
            
            # 建成年份
            if len(parts) > 4:
                year_match = re.search(r'(\d{4})', parts[4])
                listing_data['建成年份'] = int(year_match.group(1)) if year_match else None
        except NoSuchElementException:
            listing_data['户型'] = ''
            listing_data['面积'] = None
            listing_data['楼层'] = ''
            listing_data['朝向'] = ''
            listing_data['建成年份'] = None
        
        # 小区名称
        try:
            community_element = listing_element.find_element(By.CSS_SELECTOR, 'p.add_shop a')
            listing_data['小区名称'] = community_element.text
        except NoSuchElementException:
            listing_data['小区名称'] = ''
        
        # 地址
        try:
            address_element = listing_element.find_element(By.CSS_SELECTOR, 'p.add_shop span')
            listing_data['地址'] = address_element.text
        except NoSuchElementException:
            listing_data['地址'] = ''
    

        # 房主持有年份和交通信息
        try:
            label_element = listing_element.find_element(By.CSS_SELECTOR, 'p.clearfix.label')
    
            # 提取房主持有年份（没有类名的span）
            try:
                hold_year_span = label_element.find_element(By.CSS_SELECTOR, 'span:not([class])')
                listing_data['房主持有年份'] = hold_year_span.text
            except NoSuchElementException:
                listing_data['房主持有年份'] = ''
    
            # 提取交通信息
            try:
                traffic_span = label_element.find_element(By.CSS_SELECTOR, 'span.bg_none.icon_dt')
                listing_data['交通信息'] = traffic_span.text
            except NoSuchElementException:
                listing_data['交通信息'] = ''
        
        except NoSuchElementException:
            listing_data['房主持有年份'] = ''
            listing_data['交通信息'] = ''
        
        # 经纪人
        try:
            agent_element = listing_element.find_element(By.CSS_SELECTOR, 'span.people_name a')
            listing_data['经纪人'] = agent_element.text
        except NoSuchElementException:
            listing_data['经纪人'] = ''
        
        # 添加片区信息
        listing_data['片区'] = '苏州桥'
        
        return listing_data
        
    except Exception as e:
        print(f"从列表提取房源信息时出错: {e}")
        return None

In [16]:
# 初始化浏览器
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
# 主执行流程
try:
    # 1. 点击选择区域
    url_esf='https://esf.fang.com/'
    click_haidian_suzhouqiao(url_esf)
    
    # 2. 爬取房源数据
    print("开始爬取房源数据...")
    element_list = 'div.shop_list.shop_list_4 dl.clearfix'
    npage_button = 'p.last a'
    func = collect_PRICE_listing_from_list
    listings_data = scrape_listings(element_list, npage_button, func, max_pages=20)
    
    # 3. 转换为DataFrame并保存
    if listings_data:
        df = pd.DataFrame(listings_data)

        # 保存到CSV文件
        filename = 'Housing_Price_Suzhouqiao.csv'
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"数据已保存到 {filename}")
        print(f"共收集到 {len(df)} 条房源信息")
        
        # 显示前几行数据
        print("\n前5条数据预览:")
        print(df.head().to_string())
        
    else:
        print("没有收集到任何数据")
        
except Exception as e:
    print(f"主程序执行出错: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # 关闭浏览器
    driver.quit()
    print("浏览器已关闭")

已点击海淀区
已点击苏州桥片区
开始爬取房源数据...
正在爬取第 1 页...
找到 60 个房源
第 1 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 2 页...
找到 60 个房源
第 2 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 3 页...
找到 60 个房源
第 3 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 4 页...
找到 60 个房源
第 4 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 5 页...
找到 60 个房源
第 5 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 6 页...
找到 8 个房源
第 6 页收集完成，共 8 条房源
没有找到下一页，爬取结束
数据已保存到 Housing_Price_Suzhouqiao.csv
共收集到 308 条房源信息

前5条数据预览:
                             标题  总价(万元)  单价(元/平方米)    户型      面积         楼层   朝向    建成年份   小区名称            地址 房主持有年份               交通信息  经纪人   片区
0  苏州桥·友谊宾馆家属院不临街中间层正规三居室三条地铁环抱   720.0    78878.0  2室2厅   91.28   高层 （共4层）  西北向  1958.0   友谊社区  苏州桥 北三环西路47号         距4号线大兴线人民大学站约867米  张明明  苏州桥
1           苏州桥人民大学南侧三线地铁南北通透明厅   480.0    81771.0  2室1厅   58.70   中层 （共6层）  南北向  1990.0  三义庙小区     苏州桥 三义庙小区            距16号线苏州桥站约502米  张明明  苏州桥
2            苏州桥·三义庙·理工北门楼层南北两居   398.0    70194.0  2室1厅   56.70   中层 （共6层）  南北向  1985.0  三义庙小区     苏州桥 三义庙小区            距16号线苏州桥站约502米  付晓培  苏州桥
3       紫金庄园 254平4居 东西向简

##### 2.  爬取房租数据，存储在'Rent_Suzhouqiao.csv'

In [17]:
def collect_RENT_listing_from_list(listing_element):
    """从列表页元素直接收集数据"""
    listing_data = {}
    
    try:
        # 标题
        try:
            title_element = listing_element.find_element(By.CSS_SELECTOR, 'dd.info.rel p.title a')
            listing_data['标题'] = title_element.text
        except NoSuchElementException:
            listing_data['标题'] = ''
        
        # 总价
        try:
            total_price_element = listing_element.find_element(By.CSS_SELECTOR, 'dd.info.rel div.moreInfo p span.price')
            listing_data['总价'] = float(total_price_element.text)
        except NoSuchElementException:
            listing_data['总价'] = None
        
        # “租赁方式”“户型”“租赁面积”“朝向”
        try:
            tel_shop_element = listing_element.find_element(By.CSS_SELECTOR, 'p.font15.mt12.bold')
            tel_shop_text = tel_shop_element.text
            parts = [part.strip() for part in tel_shop_text.split('|')]
            
            # 租赁方式
            if len(parts) > 0:
                listing_data['租赁方式'] = parts[0]
            
            # 户型
            if len(parts) > 1:
                listing_data['户型'] = parts[1]
            
            # 租赁面积
            if len(parts) > 2:
                area_text = re.sub(r'[^\d.]', '', parts[2])
                listing_data['租赁面积'] = float(area_text)
            
            # 朝向
            if len(parts) > 3:
                listing_data['朝向'] = parts[3]
            
        except NoSuchElementException:
            listing_data['户型'] = ''
            listing_data['租赁面积'] = None
            listing_data['租赁方式'] = ''
            listing_data['朝向'] = ''
        
        # 添加片区信息
        listing_data['片区'] = '苏州桥'
        
        return listing_data
        
    except Exception as e:
        print(f"从列表提取房源信息时出错: {e}")
        return None

In [19]:
# 初始化浏览器
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
# 主执行流程
try:
    # 1. 点击选择区域
    url_rent = 'https://zu.fang.com/'
    click_haidian_suzhouqiao(url_rent)
    
    # 2. 爬取房源数据
    print("开始爬取租房房源数据...")
    element_list = 'div.houseList dl.list.hiddenMap.rel'
    npage_button = 'a[href*="/house-a015277-b02655/i32/"]'
    func = collect_RENT_listing_from_list
    listings_data = scrape_listings(element_list, npage_button, func, max_pages=20)
    
    # 3. 转换为DataFrame并保存
    if listings_data:
        df = pd.DataFrame(listings_data)

        # 保存到CSV文件
        filename = 'Rent_Suzhouqiao.csv'
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"数据已保存到 {filename}")
        print(f"共收集到 {len(df)} 条租房房源信息")
        
        # 显示前几行数据
        print("\n前5条数据预览:")
        print(df.head().to_string())
        
    else:
        print("没有收集到任何数据")
        
except Exception as e:
    print(f"主程序执行出错: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # 关闭浏览器
    driver.quit()
    print("浏览器已关闭")

已点击海淀区
已点击苏州桥片区
开始爬取租房房源数据...
正在爬取第 1 页...
找到 60 个房源
第 1 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 2 页...
找到 60 个房源
第 2 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 3 页...
找到 60 个房源
第 3 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 4 页...
找到 60 个房源
第 4 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 5 页...
找到 60 个房源
第 5 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 6 页...
找到 60 个房源
第 6 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 7 页...
找到 60 个房源
第 7 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 8 页...
找到 60 个房源
第 8 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 9 页...
找到 60 个房源
第 9 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 10 页...
找到 60 个房源
第 10 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 11 页...
找到 60 个房源
第 11 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 12 页...
找到 60 个房源
第 12 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 13 页...
找到 60 个房源
第 13 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 14 页...
找到 60 个房源
第 14 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 15 页...
找到 60 个房源
第 15 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 16 页...
找到 60 个房源
第 16 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 17 页...
找到 60 个房源
第 17 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 18 页...
找到 60 个房源
第 18 页收集完成，共 60 条房源
成功点击下一页
正在爬取第 19 页...
找到 60 个房源
第 19 页收集完成，共 60 条房源
成功点击下一页
